In [1]:
## Import the required libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import datetime as dt
import os
import imblearn
from imblearn.pipeline import make_pipeline as make_pipeline_imbfinal 
from imblearn.over_sampling import SMOTE
from imblearn.metrics import classification_report_imbalanced
from sklearn.metrics import precision_score, recall_score, f1_score, roc_auc_score, accuracy_score, classification_report
from sklearn.cross_validation import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import BernoulliNB 
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.linear_model import SGDClassifier
from sklearn.naive_bayes import GaussianNB
#from sklearn import svm
from sklearn.metrics import *
import zipfile
from zipfile import ZipFile
from io import BytesIO
import sys
import glob
import datetime
import time
import boto.s3
from boto.s3.key import Key 
import pickle
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
df = pd.read_csv('ckd.csv')

In [3]:
df_train,df_test = train_test_split(df,train_size=0.7,random_state=42)
x_train=df_train.iloc[:,:24]
y_train=df_train['Classification_ckd']
scaler.fit(x_train)
x_train_sc=scaler.transform(x_train)
x_test=df_test.iloc[:,:24]
y_test=df_test['Classification_ckd']
scaler.fit(x_test)
x_test_sc=scaler.transform(x_test)

In [4]:
accuracy =[]
model_name =[]
dataset=[]
f1score = []
precision = []
recall = []
true_positive =[]
false_positive =[]
true_negative =[]
false_negative =[]

In [5]:
logreg = LogisticRegression()
## fitiing the model
logreg.fit(x_train_sc, y_train)
filename = 'logReg_model.sav'
pickle.dump(logreg,open(filename,'wb'))
logreg

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [6]:
prediction = logreg.predict(x_train_sc)
f1 = f1_score(y_train, prediction)
p = precision_score(y_train, prediction)
r = recall_score(y_train, prediction)
a = accuracy_score(y_train, prediction)
cm = confusion_matrix(y_train, prediction)
tp = cm[0][0]
fp = cm[0][1]
fn = cm[1][0]
tn = cm[1][1]
dataset.append('Training')
model_name.append('Logistic Regression')
f1score.append(f1)
precision.append(p)
recall.append(r)
accuracy.append(a)
true_positive.append(tp) 
false_positive.append(fp)
true_negative.append(tn) 
false_negative.append(fn)
cm

array([[106,   0],
       [  0, 174]], dtype=int64)

In [7]:
feature_list = list(x_train.columns)
importances = list(logreg.coef_)
print(importances)
print(x_train.columns)

[array([ -1.40593917e-01,   3.62074941e-01,  -1.90517791e+00,
         8.14749159e-01,   5.30608385e-01,   4.69200245e-01,
         6.44831544e-02,   7.29108238e-01,  -5.04076260e-01,
        -2.68594976e-01,  -1.37051140e+00,  -8.11266712e-01,
         1.85631856e-01,  -6.70832486e-01,  -3.85156898e-01,
        -1.54641411e-03,  -2.00838374e-01,  -2.84964529e-02,
         5.95916935e-01,   7.77114539e-01,  -4.86290514e-01,
        -6.54114288e-01,   5.11615846e-01,  -2.06583000e-01])]
Index(['age', 'bp', 'sg', 'al', 'su', 'bgr', 'bu', 'sc', 'sod', 'pot', 'hemo',
       'pcv', 'wc', 'rc', 'RBC_normal', 'PC_normal', 'PCC_present',
       'BA_present', 'HTN_yes', 'DM_yes', 'CAD_yes', 'Appet_good', 'PE_yes',
       'Ane_yes'],
      dtype='object')


In [8]:
rfc = RandomForestClassifier(n_estimators=50,random_state=0)
## fitiing the model
rfc.fit(x_train_sc, y_train)
filename = 'RFC_model.sav'
pickle.dump(rfc,open(filename,'wb'))
rfc

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=50, n_jobs=1,
            oob_score=False, random_state=0, verbose=0, warm_start=False)

In [9]:
prediction = rfc.predict(x_train_sc)
f1 = f1_score(y_train, prediction)
p = precision_score(y_train, prediction)
r = recall_score(y_train, prediction)
a = accuracy_score(y_train, prediction)
cm = confusion_matrix(y_train, prediction)
tp = cm[0][0]
fp = cm[0][1]
fn = cm[1][0]
tn = cm[1][1]
model_name.append('Random Forest Classifier')
dataset.append('Training')
f1score.append(f1)
precision.append(p)
recall.append(r)
accuracy.append(a)
true_positive.append(tp) 
false_positive.append(fp)
true_negative.append(tn) 
false_negative.append(fn)
cm

array([[106,   0],
       [  0, 174]], dtype=int64)

In [10]:
feature_list = list(x_train.columns)
importances = list(rfc.feature_importances_)
feature_importances = [(x_train, round(importance, 2)) for x_train, importance in zip(feature_list, importances)]
feature_importances = sorted(feature_importances, key = lambda x: x[1], reverse = True)
[print('Variable: {:20} Importance: {}'.format(*pair)) for pair in feature_importances];

Variable: sc                   Importance: 0.18
Variable: hemo                 Importance: 0.16
Variable: pcv                  Importance: 0.14
Variable: sg                   Importance: 0.12
Variable: rc                   Importance: 0.11
Variable: al                   Importance: 0.06
Variable: DM_yes               Importance: 0.05
Variable: bu                   Importance: 0.04
Variable: sod                  Importance: 0.03
Variable: bgr                  Importance: 0.02
Variable: age                  Importance: 0.01
Variable: bp                   Importance: 0.01
Variable: su                   Importance: 0.01
Variable: pot                  Importance: 0.01
Variable: wc                   Importance: 0.01
Variable: PC_normal            Importance: 0.01
Variable: HTN_yes              Importance: 0.01
Variable: Appet_good           Importance: 0.01
Variable: PE_yes               Importance: 0.01
Variable: RBC_normal           Importance: 0.0
Variable: PCC_present          Importance

In [25]:
drop_col_list=['RBC_normal','PCC_present','BA_present','CAD_yes','Ane_yes' , 'PC_normal' , 'Appet_good' ,
               'age' , 'pot' , 'wc' , 'PE_yes' , 'bp' , 'su' ]
x_train=df_train.iloc[:,:24]
x_train.drop(drop_col_list,axis=1,inplace=True)
y_train=df_train['Classification_ckd']
scaler.fit(x_train)
x_train_sc=scaler.transform(x_train)
x_test=df_test.iloc[:,:24]
x_test.drop(drop_col_list,axis=1,inplace=True)
y_test=df_test['Classification_ckd']
scaler.fit(x_test)
x_test_sc=scaler.transform(x_test)

In [26]:
rfc = RandomForestClassifier(n_estimators=50,random_state=0)
## fitiing the model
rfc.fit(x_train_sc, y_train)
filename = 'RFC_model.sav'
pickle.dump(rfc,open(filename,'wb'))
rfc

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=50, n_jobs=1,
            oob_score=False, random_state=0, verbose=0, warm_start=False)

In [27]:
prediction = rfc.predict(x_train_sc)
f1 = f1_score(y_train, prediction)
p = precision_score(y_train, prediction)
r = recall_score(y_train, prediction)
a = accuracy_score(y_train, prediction)
cm = confusion_matrix(y_train, prediction)
tp = cm[0][0]
fp = cm[0][1]
fn = cm[1][0]
tn = cm[1][1]
model_name.append('Random Forest Classifier')
dataset.append('Training')
f1score.append(f1)
precision.append(p)
recall.append(r)
accuracy.append(a)
true_positive.append(tp) 
false_positive.append(fp)
true_negative.append(tn) 
false_negative.append(fn)
cm

array([[106,   0],
       [  0, 174]], dtype=int64)

In [28]:
feature_list = list(x_train.columns)
importances = list(rfc.feature_importances_)
feature_importances = [(x_train, round(importance, 2)) for x_train, importance in zip(feature_list, importances)]
feature_importances = sorted(feature_importances, key = lambda x: x[1], reverse = True)
[print('Variable: {:20} Importance: {}'.format(*pair)) for pair in feature_importances];

Variable: pcv                  Importance: 0.22
Variable: hemo                 Importance: 0.21
Variable: sg                   Importance: 0.13
Variable: sc                   Importance: 0.11
Variable: rc                   Importance: 0.09
Variable: al                   Importance: 0.06
Variable: DM_yes               Importance: 0.05
Variable: bgr                  Importance: 0.03
Variable: sod                  Importance: 0.03
Variable: HTN_yes              Importance: 0.03
Variable: bu                   Importance: 0.02


Conclusion : We have found after feature selection that these 11 columns have more importance and it's better to make the model with these.